<a href="https://colab.research.google.com/github/bessabra/bessabra/blob/main/cacopospro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd
from datetime import datetime, timedelta


base  = pd.read_csv('/content/drive/MyDrive/Progressão Professores/Versões v13.12/CACOPOSPRO V13.12.2.csv')
base2 = pd.read_csv('/content/drive/MyDrive/Progressão Professores/Versões v19.12/Resultados/baseAnalisev19.12.csv')


Retirada de colunas desnecessáras.

In [79]:
base2.drop('Unnamed: 0.1',  axis = 1, inplace=True)
base2.drop('Unnamed: 0',    axis = 1, inplace=True)
base2.drop('_merge',        axis = 1, inplace=True)
base2.drop('DRstatus',      axis = 1, inplace=True)
base2.drop('DR5status',     axis = 1, inplace=True)
base2.drop('REQUERIMENTO',  axis = 1, inplace=True)

base.drop('Órgão',    axis = 1, inplace=True)
base.drop('SIAPECAD', axis = 1, inplace=True)

Redefinição de tipos das variáveis.

In [80]:
base['CARGO INICIAL']     = pd.to_datetime(base['CARGO INICIAL'], errors="coerce")
base['CARGO FINAL']       = pd.to_datetime(base['CARGO FINAL'], errors="coerce")
base['VIGENCIA INICIAL']  = pd.to_datetime(base['VIGENCIA INICIAL'], errors="coerce")
base['VIGENCIA FINAL']    = pd.to_datetime(base['VIGENCIA FINAL'], errors="coerce")

base2['OBITO']                  = pd.to_datetime(base2['OBITO'], errors="coerce")
base2['INATIVIDADE']            = pd.to_datetime(base2['INATIVIDADE'], errors="coerce")
base2['DATA DO REQUERIMENTO']   = pd.to_datetime(base2['DATA DO REQUERIMENTO'], errors="coerce")
base2['DR5']                    = pd.to_datetime(base2['DR5'], errors="coerce")

Filtrando CPF, SIAPE e NOME presentes em CACOPOSPRO V13.12.2 e baseAnalisev19.12.

In [ ]:
servEncontradosCaco = base.merge(
    base2, on=['CPF', 'SIAPE', 'NOME'], 
    how='inner', 
    suffixes=['', '_'], 
    indicator=True
    )

Calculando a diferença entre DR5 e as datas de VIGENCIA INICIAL de cada Código de Carreira e Classe, por servidor. 

Em seguida, uma nova base é criada filtrada com a data e o Código de Carreira e Classe de cada servidor.

In [82]:
servEncontradosCaco['DiasDiff'] = (servEncontradosCaco['VIGENCIA INICIAL'] - servEncontradosCaco['DR5']).dt.days.abs()

servEncontradosCaco = servEncontradosCaco.sort_values(by=['NOME', 'DiasDiff'])
servEncontradosCaco = servEncontradosCaco.groupby('NOME').first().reset_index()

Calculando (DTPi - DR5), para identificar o valor da diferença e filtrar a decisão.

In [83]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year <= 1) 
    & ((servEncontradosCaco['CARGO'] == '709001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO')|(servEncontradosCaco['CARGO'] == '704001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO'))), 'REGRA'] = 'EBF <= 1'

servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year > 1) 
    & ((servEncontradosCaco['CARGO'] == '709001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO')|(servEncontradosCaco['CARGO'] == '704001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO'))), 'REGRA'] = 'EBF > 1' 

servEncontradosCaco.loc[
     ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year <= 1) 
     & ((servEncontradosCaco['CARGO'] == '707001 PROFESSOR ENS BASICO TECN TECNOL')|(servEncontradosCaco['CARGO'] == '702001 PROF DO ENSINO BASICO TEC,TECNOLOGICO'))), 'REGRA'] = 'EBTT <= 1'

servEncontradosCaco.loc[
     ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year > 1) 
     & ((servEncontradosCaco['CARGO'] == '707001 PROFESSOR ENS BASICO TECN TECNOL') |(servEncontradosCaco['CARGO'] == '702001 PROF DO ENSINO BASICO TEC,TECNOLOGICO'))), 'REGRA'] = 'EBTT > 1'

Tabelas de conversão de Código de Carreira e Classe (object) para um código de valor entre 1 - 12 (inteiro), isso facilitará o incremento.

In [ ]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-101')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-101')), 'COD_NIVEL'] = 1
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-102')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-102')), 'COD_NIVEL'] = 2
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-201')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-201')), 'COD_NIVEL'] = 3
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-202')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-202')), 'COD_NIVEL'] = 4
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-301')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-301')), 'COD_NIVEL'] = 5
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-302')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-302')), 'COD_NIVEL'] = 6
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-303')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-303')), 'COD_NIVEL'] = 7
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-304')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-304')), 'COD_NIVEL'] = 8
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-401')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-401')), 'COD_NIVEL'] = 9
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-402')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-402')), 'COD_NIVEL'] = 10
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-403')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-403')), 'COD_NIVEL'] = 11
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-404')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-404')), 'COD_NIVEL'] = 12

#--------------------------------------------------------------------------------------------------------------------------------------------

servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-101')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-101')), 'COD_NIVEL'] = 1
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-102')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-102')), 'COD_NIVEL'] = 1
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-103')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-103')), 'COD_NIVEL'] = 2
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-104')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-104')), 'COD_NIVEL'] = 2


servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-201')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-201')), 'COD_NIVEL'] = 3
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-202')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-202')), 'COD_NIVEL'] = 3
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-203')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-203')), 'COD_NIVEL'] = 4
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-204')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-204')), 'COD_NIVEL'] = 4
    
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-301')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-301')), 'COD_NIVEL'] = 5
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-302')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-302')), 'COD_NIVEL'] = 6
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-303')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-303')), 'COD_NIVEL'] = 7
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-304')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-304')), 'COD_NIVEL'] = 8
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-401')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-401')), 'COD_NIVEL'] = 9
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-402')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-402')), 'COD_NIVEL'] = 10
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-403')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-403')), 'COD_NIVEL'] = 11
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-404')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-404')), 'COD_NIVEL'] = 12

Realização do incremento de acordo com o filtro da diferença entre DTPi - DR5.


In [ ]:
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBF <= 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL'] + 3
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBF > 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL'] + 4


servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBTT <= 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL'] + 2
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBTT > 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL'] + 3

Conversão do código de valores incrementados de acordo com a regra de diferença entre DTPi - DR5.

In [ ]:
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 1), 'CNR'] = 101
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 2), 'CNR'] = 102
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 3), 'CNR'] = 201
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 4), 'CNR'] = 202
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 5), 'CNR'] = 301
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 6), 'CNR'] = 302
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 7), 'CNR'] = 303
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 8), 'CNR'] = 304
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 9), 'CNR'] = 401
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 10), 'CNR'] = 402
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 11), 'CNR'] = 403
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] >= 12), 'CNR'] = 404

Descrição do resultado da análise.

In [ ]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['CNR'] < servEncontradosCaco['NIVEL']) 
    |(servEncontradosCaco['CNR'] > servEncontradosCaco['NIVEL'])), 'RESULTADO'] = 'A DECIDIR'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] == servEncontradosCaco['NIVEL']), 'RESULTADO'] = 'INDEFERIDO'

servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] < servEncontradosCaco['NIVEL']), 'CNRstatus'] = 'O CNR calculado é MENOR que o encontrado no SIAPE'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] > servEncontradosCaco['NIVEL']), 'CNRstatus'] = 'O CNR calculado é MAIOR que o encontrado no SIAPE'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] == servEncontradosCaco['NIVEL']), 'CNRstatus'] = 'O CNR calculado é IGUAL ao encontrado no SIAPE'

Retirada de colunas desnecessáras.

In [84]:
servEncontradosCaco.drop('DESCRIÇÃO', axis = 1, inplace=True)
servEncontradosCaco.drop('CARGO_',    axis = 1, inplace=True)
servEncontradosCaco.drop('CLASSE',    axis = 1, inplace=True)
servEncontradosCaco.drop('_merge',    axis = 1, inplace=True)

Geração do xlsx com os resultados.

In [85]:
servEncontradosCaco.to_excel('ResultadoComplexo.xlsx')